In [22]:
from collections import namedtuple
import re

In [23]:
sentence = """Aspirin (ACETYLSALICYLIC Acid)   325MG TABLET  PO QD x 30 days"""
ConceptMMI = namedtuple('ConceptMMI', ['index', 'mm', 'score', 'preferred_name', 'cui', 'semtypes', 'trigger', 'location', 'pos_info', 'tree_codes'])
concept_list = []

concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='61.19', preferred_name='Aspirin', cui='C0004057', semtypes='[orch,phsu]', trigger='["ASPIRIN"-tx-1-"Aspirin"-noun-0,"Acetylsalicylic Acid"-tx-1-"ACETYLSALICYLIC Acid"-noun-0]', location='TX', pos_info='1/7;10/20', tree_codes='D02.455.426.559.389.657.410.595.176'))
concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='9.76', preferred_name='Portugal', cui='C0032729', semtypes='[geoa]', trigger='["PO"-tx-1-"PO"-noun-0]', location='TX', pos_info='48/2', tree_codes='Z01.542.727'))
concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='9.76', preferred_name='Tablet Dosage Form', cui='C0039225', semtypes='[bodm]', trigger='["TABLET"-tx-1-"TABLET"-noun-0]', location='TX', pos_info='40/6', tree_codes='D26.255.830'))
concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='3.62', preferred_name='30 days', cui='C4028326', semtypes='[tmco]', trigger='["30 days"-tx-1-"30 days"-noun-0]', location='TX', pos_info='56/7', tree_codes=''))
concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='3.45', preferred_name='Daily', cui='C0332173', semtypes='[tmco]', trigger='["QD"-tx-1-"QD"-noun-0]', location='TX', pos_info='51/2', tree_codes=''))
concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='3.45', preferred_name='Oral', cui='C0442027', semtypes='[spco]', trigger='["PO"-tx-1-"PO"-noun-0]', location='TX', pos_info='48/2', tree_codes=''))
concept_list.append(ConceptMMI(index='00000000', mm='MMI', score='3.45', preferred_name='Tablet Dosing Unit', cui='C1705223', semtypes='[qnco]', trigger='["TABLET"-tx-1-"TABLET"-noun-0]', location='TX', pos_info='40/6', tree_codes=''))

concept_list

[ConceptMMI(index='00000000', mm='MMI', score='61.19', preferred_name='Aspirin', cui='C0004057', semtypes='[orch,phsu]', trigger='["ASPIRIN"-tx-1-"Aspirin"-noun-0,"Acetylsalicylic Acid"-tx-1-"ACETYLSALICYLIC Acid"-noun-0]', location='TX', pos_info='1/7;10/20', tree_codes='D02.455.426.559.389.657.410.595.176'),
 ConceptMMI(index='00000000', mm='MMI', score='9.76', preferred_name='Portugal', cui='C0032729', semtypes='[geoa]', trigger='["PO"-tx-1-"PO"-noun-0]', location='TX', pos_info='48/2', tree_codes='Z01.542.727'),
 ConceptMMI(index='00000000', mm='MMI', score='9.76', preferred_name='Tablet Dosage Form', cui='C0039225', semtypes='[bodm]', trigger='["TABLET"-tx-1-"TABLET"-noun-0]', location='TX', pos_info='40/6', tree_codes='D26.255.830'),
 ConceptMMI(index='00000000', mm='MMI', score='3.62', preferred_name='30 days', cui='C4028326', semtypes='[tmco]', trigger='["30 days"-tx-1-"30 days"-noun-0]', location='TX', pos_info='56/7', tree_codes=''),
 ConceptMMI(index='00000000', mm='MMI', sc

In [24]:
print(sentence[9:29]) #ACETYLSALICYLIC Acid
print(sentence[47:49]) #PO
print(sentence[39:45]) #TABLET

ACETYLSALICYLIC Acid
PO
TABLET


In [25]:
for concept in concept_list:
    print(concept.preferred_name)

Aspirin
Portugal
Tablet Dosage Form
30 days
Daily
Oral
Tablet Dosing Unit


In [34]:
def view_sentence(sentence, concept_list, hide_semtypes=None):
    #close
    if hide_semtypes is None:
        hide_semtypes = ['geoa','tmco']
    
    html_str = ""
    highlight_style = """
    <style>
    span.highlight
    {
      background-color:yellow;
    }
    </style>
    """
    
    #html_str = highlight_style + "<h1 id=\"sentence\">"+ sentence + "</h1>"
    
    html_str = highlight_style + "<p>"
    
    sent_highlight = sentence
    word_concept = {}
    
    for concept in concept_list:
        #get semtypes in each concept
        tmp_1 = concept.semtypes[1:-1]     #remove '[' and ']'
        con_semtypes = re.split(',', tmp_1)
        con_semtypes = list(set(con_semtypes) - set(hide_semtypes))  #remove semtypes which is hide_semtypes

        if con_semtypes:   
            #trigger word position in sentence
            position = re.split(';', concept.pos_info)
            for pos in position:
                tmp = re.split('/', pos)
                #print(tmp)
                start_pos = int(tmp[0])
                end = int(tmp[1])


                word = sentence[start_pos-1 : start_pos-1+end]  #concept word
                if word not in word_concept:
                    word_concept[word] = []
                    word_concept[word].append(concept.preferred_name)
                else:
                    word_concept[word].append(concept.preferred_name)

    print(word_concept)
    
    for word in word_concept:
        title = ""
        
        for i in range(0, len(word_concept[word])):
            con = word_concept[word][i]
            
            for concept in concept_list:
                if concept.preferred_name == con:
                    con_id = concept.cui
                    
                    if i == len(word_concept[word])-1:
                        title += con_id + "|" + con
                    else:
                        title += con_id + "|" + con + ", "
                        #print(title)
            
        word_highlight = "<span class=\"highlight\" title=\"" + title + "\">" + word + "</span>"
        sent_highlight = sent_highlight.replace(word, word_highlight)
    
    html_str += sent_highlight
    html_str = html_str + "</p>"        
        
    Html_file = open("/Users/iir/view_sentence_html.html","w")
    Html_file.write(html_str)
    Html_file.close()

In [35]:
view_sentence(sentence, concept_list)

{'Aspirin': ['Aspirin'], 'ACETYLSALICYLIC Acid': ['Aspirin'], 'TABLET': ['Tablet Dosage Form', 'Tablet Dosing Unit'], 'PO': ['Oral']}
